In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Data Description**

You are provided with historical sales data for 45 Walmart stores located in different regions. Each store contains a number of departments, and you are tasked with predicting the department-wide sales for each store.

In addition, Walmart runs several promotional markdown events throughout the year. These markdowns precede prominent holidays, the four largest of which are the Super Bowl, Labor Day, Thanksgiving, and Christmas. The weeks including these holidays are weighted five times higher in the evaluation than non-holiday weeks. Part of the challenge presented by this competition is modeling the effects of markdowns on these holiday weeks in the absence of complete/ideal historical data.

stores.csv

This file contains anonymized information about the 45 stores, indicating the type and size of store.

train.csv

This is the historical training data, which covers to 2010-02-05 to 2012-11-01. Within this file you will find the following fields:

Store - the store number
Dept - the department number
Date - the week
Weekly_Sales -  sales for the given department in the given store
IsHoliday - whether the week is a special holiday week
test.csv

This file is identical to train.csv, except we have withheld the weekly sales. You must predict the sales for each triplet of store, department, and date in this file.

features.csv

This file contains additional data related to the store, department, and regional activity for the given dates. It contains the following fields:

Store - the store number
Date - the week
Temperature - average temperature in the region
Fuel_Price - cost of fuel in the region
MarkDown1-5 - anonymized data related to promotional markdowns that Walmart is running. MarkDown data is only available after Nov 2011, and is not available for all stores all the time. Any missing value is marked with an NA.
CPI - the consumer price index
Unemployment - the unemployment rate
IsHoliday - whether the week is a special holiday week
For convenience, the four holidays fall within the following weeks in the dataset (not all holidays are in the data):

Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13

Labor Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13

Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13

Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13

Importando as bibliotecas

In [ ]:
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import datetime
import math

# importando as libs de regressão mais populares
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor


Carregamento dos arquivos

In [ ]:
train=pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip')
stores= pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv')
features=pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip')


In [ ]:
train.head()


In [ ]:
stores.head()

In [ ]:
features.head()

Juntandos os três arquivos

In [ ]:
data = train.merge(features, on=['Store', 'Date'], how='inner').merge(stores, on=['Store'], how='inner') 
print(data.shape )

In [ ]:
data.head()

Verificando os tipos de loja

In [ ]:
sorted_type = stores.groupby('Type')
plt.style.use('ggplot')
labels=['A store','B store','C store']
sizes=sorted_type.describe()['Size'].round(1)
sizes=[(22/(6+17+22))*100,(17/(6+17+22))*100,(6/(6+17+22))*100] # converte para proporção
fig, axes = plt.subplots(1,1, figsize=(10,10))
wprops={'edgecolor':'black',
      'linewidth':2}
tprops = {'fontsize':30}
axes.pie(sizes,
        labels=labels,
        explode=(0.0,0,0),
        autopct='%1.1f%%',
        pctdistance=0.6,
        labeldistance=1.2,
        wedgeprops=wprops,
        textprops=tprops,
        radius=0.8,
        center=(0.5,0.5))
plt.show()

Veriricar o volume de vendas em feriados e não feriados.

In [ ]:
holiday = train['Weekly_Sales'].loc[train['IsHoliday']== True] # Vendas em feriados
non_holiday = train['Weekly_Sales'].loc[train['IsHoliday']== False] #vendas em não feriados.
sns.barplot(x='IsHoliday', y='Weekly_Sales', data=train)

Vendas semanas por departamento e feriadoVendas semanas por departamento e feriado


In [ ]:
data_dept = pd.concat([train['Dept'], train['Weekly_Sales'], train['IsHoliday']], axis=1)
plt.figure(figsize=(20,6))
plt.title('Box Plot Vendas semanas por departamento e feriado')
fig = sns.boxplot(x='Dept', y='Weekly_Sales', data=data_dept, showfliers=False, hue="IsHoliday")

Vendas semanas por dia e feriado


In [ ]:
data_day = pd.concat([train['Date'], train['Weekly_Sales'], train['IsHoliday']], axis=1)
plt.figure(figsize=(20,6))
plt.title('Box Plot Vendas semanais por dia e feriado')
fig = sns.boxplot(x='Date', y='Weekly_Sales', data=data_day, showfliers=False, hue='IsHoliday')


Verificando nulos

In [ ]:
data=data.fillna(0)
data.isna().sum()

Dados descritivos

In [ ]:
data.describe()

Não conseguir continuar além dessa parte.